# 和 Caffe 一样使用 im2col 实现卷积

2019 年 5 月 22 日

首先实现 im2col.


In [79]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

def make_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    B, C, H, W = input.shape
    pad = np.zeros((B, C, H + 2 * padding[0], W + 2 * padding[1]))
    pad[..., padding[0]:-padding[0], padding[1]:-padding[1]] = input
    return pad

def make_dilation(input, dilation=(1, 1)):
    if dilation == (1, 1):
        return input
    
    B, C, H, W = input.shape
    p, q = dilation
    oh, ow = p * (H - 1) + 1, q * (W - 1) + 1
    pad = np.zeros((B, C, oh, ow))
    pad[..., ::p, ::q] = input
    return pad

def unwrap_padding(input, padding=(0, 0)):
    if padding == (0, 0):
        return input
    p, q = padding
    return input[..., p:-p, q:-q]


def rotate_kernel(kernel):
    return kernel[..., ::-1, ::-1]

## 实现 im2col

输入 img 的 `shape=(N, C, h, w)`. 然后在考虑 stride 的情况下, 对图像上区域为 `img[n, :, i * s : i * s + kh, j * s : j * s + kw]` 的子块转换为行的形式.

In [7]:
def im2col(img, ksize, stride=(1, 1), dilation=(1, 1)):
    N, C, h, w = img.shape
    kh, kw = ksize
    s, _ = stride
    d, _ = dilation
    oh, ow = (h - (d * (kh - 1) + 1)) // s + 1, (w - (d * (kw - 1) + 1)) // s + 1
    x_col = np.zeros((N * oh * ow, C * kh * kw))
    for idx in range(x_col.shape[0]):
        n = idx // (oh * ow)
        i, j = (idx - (n * oh * ow)) // ow, (idx - (n * oh * ow)) % ow
        x_col[idx, :] = img[n, :, i * s : i * s + kh, j * s : j * s + kw].reshape(C * kh * kw)
    return x_col


p = 2
s = 2
d = 1

h = w = 3
kh = kw = 2
N = 1
C = 1
oC = 1
input = np.arange(N * C * h * w, dtype=np.float64).reshape(N, C, h, w)

col = im2col(input, (kh, kw))
print(col)

[[0. 1. 3. 4.]
 [1. 2. 4. 5.]
 [3. 4. 6. 7.]
 [4. 5. 7. 8.]]


## 通过 im2col 实现卷积

使用 im2col 实现卷积非常简单, 将问题转化为矩阵相乘, 先将图像转换为 `x_col`, 再将 kernel 也转换为 column 的形式

In [8]:
def conv(input, kernel, p, s, d):
    input = make_padding(input, (p, p))
    
    N, C, H, W = input.shape
    iC, oC, kh, kw = kernel.shape
    assert C == iC, 'channels not aligned'
    
    oh, ow = (H - (d * (kh - 1) + 1)) // s + 1, (W - (d * (kw - 1) + 1)) // s + 1
    
    x_col = im2col(input, (kh, kw), stride=(s, s), dilation=(d, d))
    kernel_col = kernel.transpose(1, 0, 2, 3).reshape(oC, iC * kh * kw)
    out = x_col @ kernel_col.T
    return out.reshape(N, oC, oh, ow)

## 反向传播的实现


分为误差传播和梯度更新

对于误差传播, 将 eta 中的每个值, 和 kernel 整体进行相乘, 具体的效果就是: `eta.reshape(N * oC * oh * ow, 1) * kernel.reshape(1, iC * oC * kh * kw)`, 然后使用 col2im 恢复结果, 在该函数中, 相当于 im2col 的逆过程, 但是相同位置的元素值是累加的. 此外, 注意 col2im 的参数有点多.

而对于梯度更新, 就是经过 padding 的 input 和经过 dilation(=stride rate) 后的 eta 进行卷积, 对于 kernel 中的 $w_1$, 它的梯度是 input 中所有 N 个 batch 的第一个 patch 和 eta 中所有 N 个 batch 的第 1 个输出通道进行卷积, 相当于 `input[:, ic, dh, dw]` 与 `eta[:, oc, dh, dw]` 卷积. **注意限定 input 的范围, 即对 `input[:, :, :kh - 1 + dh, :kw - 1 + dw]` 范围内的输入进行卷积**

In [95]:
def col2im(col, N, isize, channels, osize, ksize, stride=(1, 1), dilation=(1, 1)):
    h, w = isize
    oh, ow = osize
    kh, kw = ksize
    s, _ = stride
    d, _ = dilation
    iC, oC = channels

    img = np.zeros((N, iC, h, w))
    for idx in range(col.shape[0]):
        n = idx // (oC * oh * ow)
        i, j = (idx - (n * oC * oh * ow)) // ow, (idx - (n * oC * oh * ow)) % ow
        img[n, :, i * s : i * s + kh, j * s : j * s + kw] += col[idx].reshape(iC, -1, kh, kw).sum(axis=1)
    return img

def backward_conv(input, kernel, eta, p, s, d):
    N, C, h, w = input.shape
    iC, oC, kh, kw = kernel.shape
    oh, ow = eta.shape[-2:]
    input_grad = np.zeros_like(input)
    kernel_grad = np.zeros_like(kernel)
    
    input_grad = make_padding(input_grad, (p, p))
    ih, iw = input_grad.shape[-2:]
    input_grad = eta.reshape(N * oC * oh * ow, 1) * kernel.reshape(1, iC * oC * kh * kw)
    input_grad = col2im(input_grad, N, (ih, iw), (iC, oC), (oh, ow), (kh, kw), stride=(s, s), dilation=(d, d))
    input_grad = unwrap_padding(input_grad, (p, p))
    
    input = make_padding(input, (p, p))
    eta = make_dilation(eta, (s, s))
    dh, dw = eta.shape[-2:]
    x_col = im2col(input[:, :, :kh - 1 + dh, :kw - 1 + dw], (dh, dw)).reshape(N, C, -1, dh * dw).transpose(1, 2, 0, 3).reshape(-1, N * dh * dw)
    eta_col = eta.transpose(1, 0, 2, 3).reshape(oC, -1)
    kernel_grad = x_col @ eta_col.T
    kernel_grad = kernel_grad.reshape(iC, kh, kw, oC).transpose(0, 3, 1, 2)
    return input_grad, kernel_grad

## 测试结果

In [96]:
p = 3
s = 3
d = 1

h = w = 5
kh = kw = 3
N = 1
C = 1
oC = 1
input = np.arange(N * C * h * w, dtype=np.float64).reshape(N, C, h, w)
kernel = np.arange(C * oC * kh * kw, dtype=np.float64).reshape(C, oC, kh, kw)
out_conv = conv(input, kernel, p, s, d) 
print(out_conv)
eta = np.ones_like(out_conv)
backward_conv(input, kernel, eta, p, s, d)

[[[[  0.   0.   0.]
   [  0. 312. 240.]
   [  0. 304. 184.]]]]


(array([[[[0., 1., 2., 0., 1.],
          [3., 4., 5., 3., 4.],
          [6., 7., 8., 6., 7.],
          [0., 1., 2., 0., 1.],
          [3., 4., 5., 3., 4.]]]]), array([[[[36., 40., 19.],
          [56., 60., 29.],
          [23., 25., 12.]]]]))

## 使用 PyTorch 验证

In [93]:
input = Variable(torch.arange(N * C * h * w).view(N, C, h, w).float(), requires_grad=True)
net = nn.Conv2d(C, oC, (kh, kw), padding=p, stride=s, bias=False)
shape = net.weight.data.size()
net.weight.data.copy_(torch.arange(kh * kw).view(shape))
# net.weight.data.copy_(torch.ones_like(net.weight.data))
output = net(input)
print(output)
y = output.sum()
# print(y)
y.backward()
print(input.grad)
print(net.weight.grad)

tensor([[[[  0.,   0.,   0.],
          [  0., 312., 240.],
          [  0., 304., 184.]]]], grad_fn=<ThnnConv2DBackward>)
tensor([[[[0., 1., 2., 0., 1.],
          [3., 4., 5., 3., 4.],
          [6., 7., 8., 6., 7.],
          [0., 1., 2., 0., 1.],
          [3., 4., 5., 3., 4.]]]])
tensor([[[[36., 40., 19.],
          [56., 60., 29.],
          [23., 25., 12.]]]])
